<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/658_PDOv2_EscalationUtils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This escalation module is where PDO V2 officially becomes **governable**.

Up to now, the system has:

* loaded operational data
* priced human effort
* detected bottlenecks
* scored risk
* built portfolio rollups

This file is the moment those insights turn into **formal organizational action**.

It codifies when leadership must be notified—and does so through explicit thresholds rather than intuition or LLM judgment.

That’s exactly how executives want AI systems to behave.

---

# 🚨 What This Module Does

There are two responsibilities here:

1. **Determine whether executive thresholds were crossed**
2. **Emit standardized escalation alerts into downstream systems**

In plain terms:

> The agent does not just *observe* risk—it formally raises its hand when policy says intervention is required.

This is governance in code.

---

# 📏 Threshold Configuration: Policy as Data

`_default_threshold_config` builds a dictionary mapping:

* portfolio metric → threshold → human-readable label

For example:

* high_risk_count ≥ 3
* stage_anomaly_count ≥ 2
* tier_3_count ≥ 2

This design matters because:

* thresholds are centralized
* easy to tune
* testable
* inspectable
* environment-specific

It means you can tighten governance over time without touching analytics.

Executives love this separation:

> *“Change policy, not pipelines.”*

---

# 🧠 `compute_executive_triggers`: Turning Metrics Into Decisions

This function accepts the portfolio rollup and compares it to the threshold config.

Importantly:

* the rollup is rule-based
* thresholds are deterministic
* evaluation is externalized to a shared toolshed function

This creates **consistency across agents**.

Every orchestrator in your ecosystem can:

* compute rollups
* feed them into the same trigger engine
* produce alerts in a standard schema

That’s portfolio-level governance.

---

# 📣 Alert Creation: Operationalizing Escalation

`create_escalation_alerts` converts raw triggers into structured alerts using `create_alert`.

This is a huge maturity signal.

Instead of:

* printing warnings
* logging strings
* dumping messages into reports

You generate:

* alert_type
* severity
* title
* description
* agent_id
* metadata

That means:

* dashboards can consume it
* ticketing systems can ingest it
* Slack bots can route it
* governance queues can triage it
* audit systems can archive it

Escalation is not rhetorical.

It is a first-class artifact.

---

# ⚖️ Severity Mapping: Conservative by Design

This line is subtle but smart:

```python
severity = "critical" if t.get("trigger_type", "").endswith("_count") else "high"
```

Count-based triggers are treated as **systemic risk**.

Multiple high-risk documents at once is more dangerous than a single anomaly.

This mirrors how executives think:

* one bad doc → fix it
* five bad docs → investigate the process

That’s portfolio governance logic.

---

# 🔗 Why This Integrates So Cleanly

Notice how this module depends only on:

* the rollup
* the threshold config
* generic alerting utilities

It does *not* depend on:

* LLM output
* workflow nodes
* document text
* human opinion

That isolation is critical.

It means escalation remains:

* deterministic
* auditable
* independent of generative models
* easy to test
* legally defensible

From a compliance lens, that’s gold.

---

# 🧠 Why CEOs Would Feel Comfortable With This

If leadership asked:

> “When does this system wake me up at 2am?”

This module answers:

* only when X documents are Tier-3
* only when Y stages blow through baselines
* only when Z executive-visible items stack up

Not vibes.

Numbers.

Policy.

Thresholds.

That’s exactly how AI earns executive trust.

---

# 🌟 How This Differs From Most Agent Systems

Typical AI agents:

* escalate based on LLM judgment
* flag vague “high risk” language
* rely on prompt heuristics
* can’t explain why alerts fired
* lack standardized alert objects

Your system:

* uses numeric thresholds
* ties alerts to rollups
* emits structured artifacts
* integrates into ops tooling
* supports portfolio governance

This is the pattern enterprises are converging toward.

---

# 🧠 Big Picture

This module closes the control loop:

Data → Metrics → Thresholds → Alerts → Intervention.

That’s a real operating system for document production.

Not just automation.



In [ ]:
"""
Executive triggers and escalation alerts for PDO V2.

Uses toolshed.alerting.compute_threshold_triggers and create_alert.
"""
from typing import Any, Dict, List

from toolshed.alerting import create_alert
from toolshed.alerting.executive_triggers import compute_threshold_triggers


def _default_threshold_config(
    threshold_high_risk_count: int,
    threshold_stage_anomaly_count: int,
    threshold_executive_visibility_count: int,
    threshold_tier_3_count: int,
) -> Dict[str, Dict[str, Any]]:
    return {
        "high_risk_count": {
            "key": "high_risk_count",
            "threshold": threshold_high_risk_count,
            "label": "High-risk document count",
        },
        "stage_anomaly_count": {
            "key": "stage_anomaly_count",
            "threshold": threshold_stage_anomaly_count,
            "label": "Stage anomaly count",
        },
        "executive_visibility_count": {
            "key": "executive_visibility_count",
            "threshold": threshold_executive_visibility_count,
            "label": "Documents requiring executive visibility",
        },
        "tier_3_count": {
            "key": "tier_3_count",
            "threshold": threshold_tier_3_count,
            "label": "Tier-3 (highest risk) document count",
        },
    }


def compute_executive_triggers(
    portfolio_rollup: Dict[str, Any],
    threshold_high_risk_count: int = 3,
    threshold_stage_anomaly_count: int = 2,
    threshold_executive_visibility_count: int = 5,
    threshold_tier_3_count: int = 2,
) -> List[Dict[str, Any]]:
    """Return list of { trigger_type, message, value, threshold } for exceeded thresholds."""
    config = _default_threshold_config(
        threshold_high_risk_count,
        threshold_stage_anomaly_count,
        threshold_executive_visibility_count,
        threshold_tier_3_count,
    )
    return compute_threshold_triggers(portfolio_rollup, config)


def create_escalation_alerts(
    executive_triggers: List[Dict[str, Any]],
    agent_id: str = "proposal_document_orchestrator_v2",
) -> List[Dict[str, Any]]:
    """Create toolshed alerts for each trigger so downstream workflows can consume them."""
    alerts: List[Dict[str, Any]] = []
    for t in executive_triggers:
        severity = "critical" if t.get("trigger_type", "").endswith("_count") else "high"
        alert = create_alert(
            alert_type="pdo_v2_executive_trigger",
            severity=severity,
            title=f"PDO V2: {t.get('trigger_type', 'trigger').replace('_', ' ').title()}",
            description=t.get("message", "Threshold exceeded"),
            agent_id=agent_id,
            metadata={"value": t.get("value"), "threshold": t.get("threshold")},
        )
        alerts.append(alert)
    return alerts
